<a href="https://colab.research.google.com/github/TBFY/api-gateway/blob/master/notebooks/TBFY_API_Gateway.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![TBFY-Banner](https://raw.githubusercontent.com/TBFY/general/master/figures/tbfy-banner.png)

# **API Gateway**
This notebook is an example of how to use the API Gateway. The API gateway provides a flexible abstraction layer and a single entry point that securely manages communication between TBFY clients and online tools via API.
In this use case different online tools of the project will be used, via API Gateway, with the following steps:
1. Obtain an organisation identifier using the SPARQL endpoint
1. Obtain all the information of that organisation using the core API
1. Obtain all the contracting processes which this organisation has been involved using the core API
1. Obtain similar documents to the ones of the tenders which the organisation has ben involved using the search API
1. Obtain other possible names by which the organisation can be known using the reconciliation API

In [2]:
import requests
import pandas as pd
from IPython.display import display, HTML

# **Step 1**
Using the resource **/triples**, the API Gateway will connect to the SPARQL endpoint to get an identifier according to the chosen name parameter

# **Step 2**
Using the resource **/organisation/{id}**, the API Gateway will connect to the core API to extract all the information about the identifier previously obtained

In [ ]:
#@title Information of an Organisation

id =  "fr-130003981"#@param {type:"string"}

params = 'http://tbfy.librairy.linkeddata.es/api/organisation/' + id
print ('Requesting : ' + params )
print()
resp = requests.get(params)
if resp.status_code != 200:
    # This means something went wrong.
    raise ApiError('GET /tasks/ {}'.format(resp.status_code))

lista=[]
for items in resp.json():
  lista.append(resp.json())
i=0
df = pd.DataFrame(columns=['legalName', 'organisation', 'employees', 'founding_date', 'activity', 'address', 'postal_code', 'locality'])
for todo_item in lista:
  df.loc[i] = [todo_item['legalName'],todo_item['id'],todo_item['numberOfEmployees'],todo_item['foundingDate'],todo_item['activity'],todo_item['fullAddress']['address'],todo_item['fullAddress']['postCode'],todo_item['fullAddress']['postName']]

display(HTML(df.to_html(justify='left')))

# **Step 3**
Using the resource **/organisation/{id}/contractingProcess**, the API Gateway will connect to the core API to show all the contracting process which the organisation has been involved.

In [ ]:
#@title Contacting processes of an organisation

params = 'http://tbfy.librairy.linkeddata.es/api/organisation/' + id + '/contractingProcess'
print ('Requesting : ' + params )
print()
resp = requests.get(params)

if resp.status_code != 200:
    # This means something went wrong.
    raise ApiError('GET /tasks/ {}'.format(resp.status_code))

i=0
df = pd.DataFrame(columns=['contracting_process_id'])
for todo_item in resp.json():
  df.loc[i] = [todo_item['id']]
  i+=1

display(HTML(df.to_html(justify='left')))

# **Step 4**
Using the resource **documents/{id}/items**, the API Gateway will connect to the search API to get similar documents to the ones of the tenders which the organisation has ben involved

In [ ]:
#@title List of tenders of an organisation and related documents

cp_id =  "ocds-0c46vo-0133-000002-2019"#@param {type:"string"}

params = 'http://tbfy.librairy.linkeddata.es/api/contractingProcess/' + cp_id + '/tender'
print ('Requesting : ' + params )
print()
resp = requests.get(params)

if resp.status_code != 200:
    # This means something went wrong.
    raise ApiError('GET /tasks/ {}'.format(resp.status_code))

i=0
df = pd.DataFrame(columns=['tender_id','title','description','status'])
for todo_item in resp.json():
  df.loc[i] = [todo_item['id'],todo_item['title'],todo_item['description'],todo_item['status']]
  i+=1

display(HTML(df.to_html(justify='left')))
print()
print()

print("***** RELATED DOCUMENTS *****")
print()
# Documents
#doc_id = todo_item['id']
doc_id = 'ocds-0c46vo-0096-JAN156235_JAN156235'

params = 'http://tbfy.librairy.linkeddata.es/api/documents/' + doc_id + '/items'
print ('Requesting : ' + params )
print()
resp = requests.get(params)

if resp.status_code != 200:
    # This means something went wrong.
    raise ApiError('GET /tasks/ {}'.format(resp.status_code))

i=0
df = pd.DataFrame(columns=['document','name','score'])
for todo_item in resp.json():
  df.loc[i] = [todo_item['id'],todo_item['name'],todo_item['score']]
  i+=1

display(HTML(df.to_html(justify='left')))

# **Step 5**
Using the resource **/brands**, the API Gateway will connect to the reconciliation API to show other possible names by which the organisation can be known

In [ ]:
#@title Reconcile Organisations

name_org =  ""#@param {type:"string"}
newname= name_org.replace(" ", "+")
jurisdiction = "gb" #@param ["gb", "es", "fr", "it", "pt", "no"] {type:"string"}  

params = 'https://opencorporates.com/reconcile/' + jurisdiction + '?query=' + newname
resp = requests.get(params)

print ('Requesting : ' + params )
print ()
json_val = resp.json()['result']

if resp.status_code != 200:
    # This means something went wrong.
    raise ApiError('GET /tasks/ {}'.format(resp.status_code))

df = pd.DataFrame(columns=['id', 'name', 'score', 'match', 'uri'])
i=0
for todo_item in json_val:
  new_id = todo_item['id'].replace("/companies/","")
  new_id = new_id.replace("/","-")
  df.loc[i] = [new_id, todo_item['name'], todo_item['score'], todo_item['match'], todo_item['uri']]
  i+=1

display(HTML(df.to_html(justify='left')))
